## Tensor

### 起步：numpy

介绍 PyTorch 之前，首先使用 numpy 实现网络。

numpy 提供了n维数组，许多操作数组的函数。它是科学计算的通用框架，它对计算图、深度学习或梯度一无所知，但可手动实现网络的前向传播和反向传播，轻松使用两层网络拟合随机数据。

In [1]:
# -*- coding: utf-8 -*-
import numpy as np

# N是数据样本数，D_in是输入层维度
# H是隐藏层维度，D_out是输出层维度
N, D_in, H, D_out = 64, 1000, 100, 10

# 构建随机输入和输出数据
x = np.random.randn(N, D_in)
y = np.random.randn(N, D_out)

# 随机初始化权重
w1 = np.random.randn(D_in, H)
w2 = np.random.randn(H, D_out)

learning_rate = 1e-6  # 学习率
iteration_count = 500  # 迭代次数
for t in range(iteration_count):
    # 前向传播：计算预测值 y
    h = x.dot(w1)  # dot矩阵乘法
    h_relu = np.maximum(h, 0)  # h = ReLU(h)
    y_pred = h_relu.dot(w2) # 注意，输出层没有应用 ReLU

    # 计算并输出损失
    loss = np.square(y_pred - y).sum()  # 平方损失
    print("iteration", t, ",", "loss:", loss)  # t是循环变量

    # 反向传播，计算w1和w2关于损失的梯度
    grad_y_pred = 2.0 * (y_pred - y)
    grad_w2 = h_relu.T.dot(grad_y_pred)
    grad_h_relu = grad_y_pred.dot(w2.T)
    grad_h = grad_h_relu.copy()
    grad_h[h < 0] = 0
    grad_w1 = x.T.dot(grad_h)

    # 更新权重
    w1 -= learning_rate * grad_w1
    w2 -= learning_rate * grad_w2

iteration 0 , loss: 39019361.38700613
iteration 1 , loss: 35719082.139826335
iteration 2 , loss: 32509352.125539895
iteration 3 , loss: 25420309.580064043
iteration 4 , loss: 16606637.71162427
iteration 5 , loss: 9387762.619381998
iteration 6 , loss: 5149326.868747448
iteration 7 , loss: 3007308.489897264
iteration 8 , loss: 1967574.1297512027
iteration 9 , loss: 1423981.9169151098
iteration 10 , loss: 1104667.3536703452
iteration 11 , loss: 892595.5990871357
iteration 12 , loss: 738092.2728450412
iteration 13 , loss: 618807.9198511755
iteration 14 , loss: 523574.3970470655
iteration 15 , loss: 446008.78058524156
iteration 16 , loss: 382001.6517942947
iteration 17 , loss: 328756.84876854956
iteration 18 , loss: 284160.2632113299
iteration 19 , loss: 246638.5445840901
iteration 20 , loss: 214840.1214017758
iteration 21 , loss: 187770.00418735202
iteration 22 , loss: 164686.3088019176
iteration 23 , loss: 144847.86762839375
iteration 24 , loss: 127742.92794579946
iteration 25 , loss: 112

iteration 229 , loss: 0.11273381354723264
iteration 230 , loss: 0.10688977878647675
iteration 231 , loss: 0.10135500967126455
iteration 232 , loss: 0.09611211989963034
iteration 233 , loss: 0.09114553818302557
iteration 234 , loss: 0.0864406087379351
iteration 235 , loss: 0.08198331994283264
iteration 236 , loss: 0.07776001519745812
iteration 237 , loss: 0.07375798508108763
iteration 238 , loss: 0.06996604906194312
iteration 239 , loss: 0.06637264694039033
iteration 240 , loss: 0.06296701063977775
iteration 241 , loss: 0.05974011369023456
iteration 242 , loss: 0.05668193799372451
iteration 243 , loss: 0.05378212789565301
iteration 244 , loss: 0.051033747469932916
iteration 245 , loss: 0.04842810248894612
iteration 246 , loss: 0.04595770987643143
iteration 247 , loss: 0.04361559420791622
iteration 248 , loss: 0.041395044060547365
iteration 249 , loss: 0.039289669743393804
iteration 250 , loss: 0.037293242026212356
iteration 251 , loss: 0.035399941226231794
iteration 252 , loss: 0.033604

iteration 455 , loss: 1.7445935378968276e-06
iteration 456 , loss: 1.6655665130788384e-06
iteration 457 , loss: 1.5901395882884578e-06
iteration 458 , loss: 1.5181475796990957e-06
iteration 459 , loss: 1.4494346177532335e-06
iteration 460 , loss: 1.3838473597522563e-06
iteration 461 , loss: 1.3212450847941512e-06
iteration 462 , loss: 1.2614895774650485e-06
iteration 463 , loss: 1.2044554949853435e-06
iteration 464 , loss: 1.1500106105274685e-06
iteration 465 , loss: 1.0980530434214043e-06
iteration 466 , loss: 1.0484605002834046e-06
iteration 467 , loss: 1.0011062040930382e-06
iteration 468 , loss: 9.559031193539084e-07
iteration 469 , loss: 9.127490726000991e-07
iteration 470 , loss: 8.715570044647074e-07
iteration 471 , loss: 8.322321991994698e-07
iteration 472 , loss: 7.94690115659629e-07
iteration 473 , loss: 7.588505356090387e-07
iteration 474 , loss: 7.246352641455797e-07
iteration 475 , loss: 6.919722158897731e-07
iteration 476 , loss: 6.607873267072988e-07
iteration 477 , loss

## Pytorch: Tensor

Numpy 是一个很棒的框架，但它不能利用 GPU 来加速其数值计算。对于现代深度神经网络，GPU通常提供50倍或更高的加速，所以不幸的是，numpy 对于现代深度学习来说还不够。

在这里，我们介绍最基本的 PyTorch 概念：Tensor。 PyTorch Tensor 在概念上与 numpy 数组相同：Tensor 是一个n维数组，PyTorch 提供了许多用于在这些 Tensor 上运算的函数。Tensor 可以跟踪计算图和梯度，但它们也可用作科学计算的通用工具。

与 numpy 不同，PyTorch Tensor 可以利用 GPU 加速其数值计算。要在 GPU 上运行 PyTorch Tensor，只需将其转换为新的数据类型即可。

在这里，我们使用 PyTorch Tensor 将双层网络与随机数据相匹配。像上面的 numpy 示例一样，我们需要手动实现通过网络的前向和后向传播：

In [2]:
# -*- coding: utf-8 -*-

import torch

dtype = torch.float  # data type
device = torch.device("cpu")
# device = torch.device("cuda:0") # 撤销这句注释以在 GPU 上运行

# N是数据样本数，D_in是输入层维度
# H是隐藏层维度，D_out是输出层维度
N, D_in, H, D_out = 64, 1000, 100, 10

# 创建随机输入和输出数据
x = torch.randn(N, D_in, device=device, dtype=dtype)
y = torch.randn(N, D_out, device=device, dtype=dtype)

# 随机初始化权重
w1 = torch.randn(D_in, H, device=device, dtype=dtype)
w2 = torch.randn(H, D_out, device=device, dtype=dtype)

learning_rate = 1e-6  # 学习率
iteration_count = 500  # 迭代次数
for t in range(iteration_count):
    # 前向传播：计算预测值y
    h = x.mm(w1)  # 矩阵乘法 matrix multiply
    h_relu = h.clamp(min=0)  # h_relu = ReLU(h)
    # 函数 clamp
    # if x < min: x = min
    # else if x > max: x = max
    # else x = x
    y_pred = h_relu.mm(w2)  # 矩阵乘法

    # 计算并输出损失
    loss = (y_pred - y).pow(2).sum().item()  # 平方损失
    print("iteration", t, ",", "loss:", loss)  # t是循环变量

    # 反向传播，计算w1和w2关于损失的梯度
    grad_y_pred = 2.0 * (y_pred - y)
    grad_w2 = h_relu.t().mm(grad_y_pred)
    grad_h_relu = grad_y_pred.mm(w2.t())
    grad_h = grad_h_relu.clone()
    grad_h[h < 0] = 0
    grad_w1 = x.t().mm(grad_h)

    # 更新权重
    w1 -= learning_rate * grad_w1
    w2 -= learning_rate * grad_w2

iteration 0 , loss: 30581504.0
iteration 1 , loss: 27329054.0
iteration 2 , loss: 25768940.0
iteration 3 , loss: 22976022.0
iteration 4 , loss: 18152610.0
iteration 5 , loss: 12641956.0
iteration 6 , loss: 7968243.5
iteration 7 , loss: 4815052.5
iteration 8 , loss: 2941822.0
iteration 9 , loss: 1893403.75
iteration 10 , loss: 1305025.75
iteration 11 , loss: 961672.8125
iteration 12 , loss: 747942.9375
iteration 13 , loss: 604822.4375
iteration 14 , loss: 502382.03125
iteration 15 , loss: 424860.53125
iteration 16 , loss: 363749.1875
iteration 17 , loss: 314159.625
iteration 18 , loss: 273175.90625
iteration 19 , loss: 238828.546875
iteration 20 , loss: 209776.625
iteration 21 , loss: 185002.421875
iteration 22 , loss: 163748.921875
iteration 23 , loss: 145438.828125
iteration 24 , loss: 129555.3125
iteration 25 , loss: 115736.7109375
iteration 26 , loss: 103673.5546875
iteration 27 , loss: 93098.7109375
iteration 28 , loss: 83802.4375
iteration 29 , loss: 75611.21875
iteration 30 , los

iteration 231 , loss: 2.31713604927063
iteration 232 , loss: 2.2256031036376953
iteration 233 , loss: 2.137796401977539
iteration 234 , loss: 2.0533764362335205
iteration 235 , loss: 1.9725390672683716
iteration 236 , loss: 1.894963026046753
iteration 237 , loss: 1.8202667236328125
iteration 238 , loss: 1.7485911846160889
iteration 239 , loss: 1.6797354221343994
iteration 240 , loss: 1.6136040687561035
iteration 241 , loss: 1.5502310991287231
iteration 242 , loss: 1.489249348640442
iteration 243 , loss: 1.4307622909545898
iteration 244 , loss: 1.3745924234390259
iteration 245 , loss: 1.3207014799118042
iteration 246 , loss: 1.2688047885894775
iteration 247 , loss: 1.2189857959747314
iteration 248 , loss: 1.1712957620620728
iteration 249 , loss: 1.125312089920044
iteration 250 , loss: 1.0812067985534668
iteration 251 , loss: 1.0388457775115967
iteration 252 , loss: 0.9981720447540283
iteration 253 , loss: 0.9591154456138611
iteration 254 , loss: 0.9215317368507385
iteration 255 , loss: 

iteration 486 , loss: 0.00028901596670039
iteration 487 , loss: 0.00028307060711085796
iteration 488 , loss: 0.0002761351934168488
iteration 489 , loss: 0.0002704347134567797
iteration 490 , loss: 0.0002648920635692775
iteration 491 , loss: 0.0002589305804576725
iteration 492 , loss: 0.0002539852575864643
iteration 493 , loss: 0.00024899072013795376
iteration 494 , loss: 0.0002441420219838619
iteration 495 , loss: 0.00023855161271058023
iteration 496 , loss: 0.0002345659158891067
iteration 497 , loss: 0.00022961907961871475
iteration 498 , loss: 0.00022488643298856914
iteration 499 , loss: 0.00022019920288585126


## 自动微分

### PyTorch: Tensor 和自动微分

在上面的例子中，我们手动实现了神经网络的前向和后向传播。手动实现反向传播对于小型双层网络来说并非难题，但对于大型复杂网络来说，很快就会变得非常繁琐。

值得庆幸的是，我们可以使用自动微分来自动计算神经网络中的后向传播。PyTorch 中的 autograd 包提供了这个功能。使用 autograd 时，网络的前向传播将定义计算图；图中的节点是 Tensor，边是从输入 Tensor 产生输出 Tensor 的函数。通过此图反向传播，您可以轻松计算梯度。

这听起来很复杂，在实践中使用起来非常简单。每个 Tensor 代表计算图中的节点。如果 x 是具有 x.requires_grad = True 的 Tensor，则 x.grad 是另一个Tensor，相对于某个标量值保持 x 的梯度。

在这里，我们使用 PyTorch Tensor 和 autograd 来实现我们的双层网络;现在我们不再需要手动实现通过网络的反向传播：

In [3]:
# -*- coding: utf-8 -*-
import torch

dtype = torch.float
device = torch.device("cpu")
# device = torch.device("cuda:0") # 取消注释以在 GPU 上运行

# N是数据样本数，D_in是输入层维度
# H是隐藏层维度，D_out是输出层维度
N, D_in, H, D_out = 64, 1000, 100, 10

# 创建随机输入和输出数据
# 设置 requires_grad = False 表示在反向传播中不需要计算关于该 Tensor 的梯度
x = torch.randn(N, D_in, device=device, dtype=dtype)
y = torch.randn(N, D_out, device=device, dtype=dtype)

# 随机初始化权重
# 设置 requires_grad = True 表示在反向传播中想要计算关于该 Tensor 的梯度
w1 = torch.randn(D_in, H, device=device, dtype=dtype, requires_grad=True)
w2 = torch.randn(H, D_out, device=device, dtype=dtype, requires_grad=True)

learning_rate = 1e-6  # 学习率
iteration_count = 500  # 迭代次数
for t in range(iteration_count):
    # 前向传播：计算预测值y
    y_pred = x.mm(w1).clamp(min=0).mm(w2)
    # 因为反向传播不再需要手动实现，不需要定义中间变量

    # 使用 Tensor 运算，计算并输出损失
    # 损失是一个 Tensor (1,)
    # loss.item() 获得 loss 中的标量
    loss = (y_pred - y).pow(2).sum()  # 平方损失
    print("iteration", t, ",", "loss:", loss.item())  # t是循环变量

    # 使用 autograd 计算反向传播
    # 这条语句将计算损失函数关于所有 requires_grad = True 的 Tensor 的梯度
    # 之后，得到 w1.grad 和 w2.grad
    loss.backward()

    # 使用梯度下降手动更新权重
    # 包在 torch.no_grad() 是因为权重的 requires_grad = True，但此时不需要追踪梯度
    # 另一种方式是操作 weight.data 和 weight.grad.data
    # 回想一下，tensor.data给出了一个 Tensor ，它与 Tensor 共享存储，但不跟踪历史记录，即没有梯度的产生
    # 也可以使用 torch.optim.SGD 实现
    with torch.no_grad():
        w1 -= learning_rate * w1.grad
        w2 -= learning_rate * w2.grad

        # 更新权重后手动清零梯度
        w1.grad.zero_()
        w2.grad.zero_()

iteration 0 , loss: 28531214.0
iteration 1 , loss: 27689062.0
iteration 2 , loss: 30838962.0
iteration 3 , loss: 33524800.0
iteration 4 , loss: 31596386.0
iteration 5 , loss: 24105248.0
iteration 6 , loss: 14803715.0
iteration 7 , loss: 7824204.5
iteration 8 , loss: 3994001.25
iteration 9 , loss: 2196424.25
iteration 10 , loss: 1377377.375
iteration 11 , loss: 980553.3125
iteration 12 , loss: 763403.25
iteration 13 , loss: 626416.5
iteration 14 , loss: 529183.0625
iteration 15 , loss: 454337.65625
iteration 16 , loss: 393925.46875
iteration 17 , loss: 343872.4375
iteration 18 , loss: 301695.5
iteration 19 , loss: 265823.03125
iteration 20 , loss: 235087.296875
iteration 21 , loss: 208601.359375
iteration 22 , loss: 185668.6875
iteration 23 , loss: 165714.65625
iteration 24 , loss: 148287.453125
iteration 25 , loss: 133008.921875
iteration 26 , loss: 119610.984375
iteration 27 , loss: 107816.296875
iteration 28 , loss: 97388.1953125
iteration 29 , loss: 88142.0703125
iteration 30 , loss

iteration 256 , loss: 0.32217103242874146
iteration 257 , loss: 0.30832892656326294
iteration 258 , loss: 0.2950286567211151
iteration 259 , loss: 0.2823435962200165
iteration 260 , loss: 0.27019017934799194
iteration 261 , loss: 0.2585216462612152
iteration 262 , loss: 0.2474524825811386
iteration 263 , loss: 0.23679931461811066
iteration 264 , loss: 0.22663022577762604
iteration 265 , loss: 0.21690930426120758
iteration 266 , loss: 0.20761583745479584
iteration 267 , loss: 0.19870463013648987
iteration 268 , loss: 0.190194770693779
iteration 269 , loss: 0.18205435574054718
iteration 270 , loss: 0.17425155639648438
iteration 271 , loss: 0.16680128872394562
iteration 272 , loss: 0.15961702167987823
iteration 273 , loss: 0.1527835875749588
iteration 274 , loss: 0.14627139270305634
iteration 275 , loss: 0.1400095522403717
iteration 276 , loss: 0.13401833176612854
iteration 277 , loss: 0.1282569020986557
iteration 278 , loss: 0.12282818555831909
iteration 279 , loss: 0.11758533120155334
i

iteration 478 , loss: 0.0001502787199569866
iteration 479 , loss: 0.00014739845937583596
iteration 480 , loss: 0.0001449470000807196
iteration 481 , loss: 0.00014216073032002896
iteration 482 , loss: 0.0001400971959810704
iteration 483 , loss: 0.00013714835222344846
iteration 484 , loss: 0.00013471576676238328
iteration 485 , loss: 0.00013280218990985304
iteration 486 , loss: 0.00013051132555119693
iteration 487 , loss: 0.0001284302561543882
iteration 488 , loss: 0.0001263170561287552
iteration 489 , loss: 0.00012421308201737702
iteration 490 , loss: 0.00012221717042848468
iteration 491 , loss: 0.00012026918557239696
iteration 492 , loss: 0.00011832278687506914
iteration 493 , loss: 0.00011648976942524314
iteration 494 , loss: 0.00011444505071267486
iteration 495 , loss: 0.00011253806587774307
iteration 496 , loss: 0.00011076083319494501
iteration 497 , loss: 0.00010893803846556693
iteration 498 , loss: 0.0001069352074409835
iteration 499 , loss: 0.00010499829659238458


### 定义新的自动微分函数

在这之后，每个原始 autograd 运算符实际上是两个在 Tensor 上运行的函数。前向函数从输入 Tensor 计算输出 Tensor. 反向函数接收输出 Tensor 相对于某个标量值的梯度，并根据相同的标量值计算输入 Tensor 的梯度。

在 PyTorch 中，我们可以通过定义 torch.autograd.Function 的子类并实现前向和后向函数来轻松定义我们自己的 autograd 运算符。然后我们可以使用我们的新 autograd 运算符，通过构造一个实例并像函数一样调用它，传递包含输入数据的 Tensor。

在这个例子中，我们定义了自己的自定义 autograd 函数来执行 ReLU 非线性，并使用它来实现我们的双层网络：

In [4]:
# -*- coding: utf-8 -*-
import torch


class MyReLU(torch.autograd.Function):
    """
    我们可以通过继承 torch.autograd.Function 并实现前向和后向传播函数来实现自定义的 autograd 函数
    """

    @staticmethod
    def forward(ctx, input):
        """
        在前向传播中，我们接收包含输入的 Tensor 并返回包含输出的 Tensor 
        ctx是一个上下文对象，可用于存储信息以进行反向计算
        您可以使用 ctx.save_for_backward 方法缓存任意对象以在反向传播中使用
        """
        ctx.save_for_backward(input)
        return input.clamp(min=0)  # if x < 0 : x = 0

    @staticmethod
    def backward(ctx, grad_output):
        """
        在反向传播中，我们接收含有损失关于输出的梯度的 Tensor，
        并且我们需要计算损失关于输入的梯度。
        """
        input, = ctx.saved_tensors
        grad_input = grad_output.clone()
        grad_input[input < 0] = 0
        return grad_input


dtype = torch.float
device = torch.device("cpu")
# device = torch.device("cuda:0") # 取消注释以在 GPU 上运行

# N是数据样本数，D_in是输入层维度
# H是隐藏层维度，D_out是输出层维度
N, D_in, H, D_out = 64, 1000, 100, 10

# 创建随机输入和输出数据
x = torch.randn(N, D_in, device=device, dtype=dtype)
y = torch.randn(N, D_out, device=device, dtype=dtype)

# 随机初始化权重
w1 = torch.randn(D_in, H, device=device, dtype=dtype, requires_grad=True)
w2 = torch.randn(H, D_out, device=device, dtype=dtype, requires_grad=True)

learning_rate = 1e-6  # 学习率
iteration_count = 500  # 迭代次数
for t in range(iteration_count):
    # 为应用自定义的函数，取假名 relu
    relu = MyReLU.apply
    
    # 前向传播：计算预测值y，使用自定义函数
    y_pred = relu(x.mm(w1)).mm(w2)
    
    # 计算并打印损失
    loss = (y_pred - y).pow(2).sum()
    print("iteration", t, ",", "loss:", loss.item())  # t是循环变量
    
    # 使用 autograd 计算反向传播
    loss.backward()
    
    # 使用梯度下降更新权重
    with torch.no_grad():
        w1 -= learning_rate * w1.grad
        w2 -= learning_rate * w2.grad
        
        # 更新权重后手动清零梯度
        w1.grad.zero_()
        w2.grad.zero_()

iteration 0 , loss: 30721902.0
iteration 1 , loss: 28507020.0
iteration 2 , loss: 33064666.0
iteration 3 , loss: 38874488.0
iteration 4 , loss: 39144912.0
iteration 5 , loss: 30448758.0
iteration 6 , loss: 17675790.0
iteration 7 , loss: 8324366.5
iteration 8 , loss: 3812623.25
iteration 9 , loss: 2011797.625
iteration 10 , loss: 1294538.25
iteration 11 , loss: 967610.0625
iteration 12 , loss: 784708.5625
iteration 13 , loss: 661807.875
iteration 14 , loss: 568817.9375
iteration 15 , loss: 494109.34375
iteration 16 , loss: 432066.53125
iteration 17 , loss: 379756.96875
iteration 18 , loss: 335191.5625
iteration 19 , loss: 296913.84375
iteration 20 , loss: 263870.71875
iteration 21 , loss: 235224.5625
iteration 22 , loss: 210266.609375
iteration 23 , loss: 188481.25
iteration 24 , loss: 169364.546875
iteration 25 , loss: 152546.765625
iteration 26 , loss: 137704.984375
iteration 27 , loss: 124556.578125
iteration 28 , loss: 112884.0859375
iteration 29 , loss: 102512.140625
iteration 30 ,

iteration 218 , loss: 3.0276153087615967
iteration 219 , loss: 2.896275281906128
iteration 220 , loss: 2.7707016468048096
iteration 221 , loss: 2.6505415439605713
iteration 222 , loss: 2.535844326019287
iteration 223 , loss: 2.4260377883911133
iteration 224 , loss: 2.3211584091186523
iteration 225 , loss: 2.2208757400512695
iteration 226 , loss: 2.125004529953003
iteration 227 , loss: 2.033344030380249
iteration 228 , loss: 1.9456048011779785
iteration 229 , loss: 1.861838936805725
iteration 230 , loss: 1.7819498777389526
iteration 231 , loss: 1.705227017402649
iteration 232 , loss: 1.6319055557250977
iteration 233 , loss: 1.5616639852523804
iteration 234 , loss: 1.4946904182434082
iteration 235 , loss: 1.4305580854415894
iteration 236 , loss: 1.3692626953125
iteration 237 , loss: 1.3106790781021118
iteration 238 , loss: 1.254668951034546
iteration 239 , loss: 1.2009577751159668
iteration 240 , loss: 1.1496418714523315
iteration 241 , loss: 1.1005030870437622
iteration 242 , loss: 1.05

iteration 425 , loss: 0.0007699169800616801
iteration 426 , loss: 0.0007504752138629556
iteration 427 , loss: 0.000730007654055953
iteration 428 , loss: 0.0007089725695550442
iteration 429 , loss: 0.0006897827843204141
iteration 430 , loss: 0.0006708703003823757
iteration 431 , loss: 0.0006533346604555845
iteration 432 , loss: 0.000634866941254586
iteration 433 , loss: 0.0006191569264046848
iteration 434 , loss: 0.0006030915537849069
iteration 435 , loss: 0.0005871151806786656
iteration 436 , loss: 0.0005725636729039252
iteration 437 , loss: 0.000557442253921181
iteration 438 , loss: 0.0005421680398285389
iteration 439 , loss: 0.0005287661915645003
iteration 440 , loss: 0.0005143327871337533
iteration 441 , loss: 0.0005019673844799399
iteration 442 , loss: 0.000488889345433563
iteration 443 , loss: 0.00047647891915403306
iteration 444 , loss: 0.000464641023427248
iteration 445 , loss: 0.00045404996490105987
iteration 446 , loss: 0.0004423176287673414
iteration 447 , loss: 0.00043155139

### TensorFlow: 静态图

PyTorch autograd 看起来很像 TensorFlow：在两个框架中我们定义了一个计算图，并使用自动微分来计算梯度。两者之间最大的区别是TensorFlow 的计算图是静态的，PyTorch 使用动态计算图。

在TensorFlow中，我们定义计算图一次，然后一遍又一遍地执行相同的图，可能将不同的输入数据提供给图。在 PyTorch 中，每个前向传播定义了一个新的计算图。

静态图很好，因为你可以预先优化图；例如，框架可能决定融合某些图操作以提高效率，或者提出一种策略，用于在多个 GPU 或许多机器上分布图。如果您反复使用相同的图表，那么这个代价可能高昂的前期优化可以分摊，因为相同的图表会反复重新运行。

静态和动态图的一个不同之处在于控制流程。对于某些模型，我们可能希望对每个数据点执行不同的计算；例如，可以针对每个数据点针对不同数量的时间步长展开循环网络；这种展开可以作为循环实现。使用静态图形，循环结构需要是图的一部分；因此，TensorFlow 提供了诸如 tf.scan 之类的运算符，用于将循环嵌入到图中。使用动态图情况更简单：因为我们为每个示例动态构建图，我们可以使用常规命令流程控制来执行每个输入不同的计算。

与上面的 PyTorch autograd 示例相比，这里我们使用 TensorFlow 来拟合一个简单的双层网：

In [5]:
# -*- coding: utf-8 -*-
import tensorflow as tf
import numpy as np

# 首先设置计算图：

# N是数据样本数，D_in是输入层维度
# H是隐藏层维度，D_out是输出层维度
N, D_in, H, D_out = 64, 1000, 100, 10

# 为输入和目标数据创建占位符，在执行图时占位符会被填充数据
x = tf.placeholder(tf.float32, shape=(None, D_in))
y = tf.placeholder(tf.float32, shape=(None, D_out))
# 占位符 shape 没有包含数据样本数

# 为权重创建变量并使用随机数据初始化
# 一个 TensorFlow 变量在计算图执行的过程中保持其值。
w1 = tf.Variable(tf.random_normal((D_in, H)))
w2 = tf.Variable(tf.random_normal((H, D_out)))

# 前向传播：使用 TensorFlow Tensor 计算预测值y
# 注意这些代码并没有实际执行任何数值操作，它仅设置稍后将执行的计算图

h = tf.matmul(x, w1)  # 矩阵乘法
h_relu = tf.maximum(h, tf.zeros(1))
y_pred = tf.matmul(h_relu, w2)

# 使用 TensorFlow Tensor 的运算计算损失
loss = tf.reduce_sum((y - y_pred)**2.0)

# 损失函数关于 w1 和 w2 的梯度
grad_w1, grad_w2 = tf.gradients(loss, [w1, w2])

# 使用梯度下降更新权重
# 要实际更新权重，我们需要在执行计算图时评估 new_w1 和 new_w2
# 请注意，在 TensorFlow 中，更新权重值的行为是计算图的一部分
# 在 PyTorch 中，这发生在计算图之外
learning_rate = 1e-6
new_w1 = w1.assign(w1 - learning_rate * grad_w1)
new_w2 = w2.assign(w2 - learning_rate * grad_w2)

# 现在已经构建了计算图，进入 TensorFlow session 以执行计算图
with tf.Session() as sess:
    # 运行图，初始化权重 w1 和 w2
    sess.run(tf.global_variables_initializer())

    # 创建包含输入 x 和目标 y 的实际数据的 numpy 数组
    x_value = np.random.randn(N, D_in)
    y_value = np.random.randn(N, D_out)
    
    iteration_count = 500  # 迭代次数
    
    # 每次循环将 x_value 绑定到 x，y_value 绑定到 y
    # 每次循环计算损失与新权重
    for t in range(iteration_count):
        loss_value, _, _ = sess.run(
            [loss, new_w1, new_w2], feed_dict={
                x: x_value,
                y: y_value
            })
        print(t,loss_value)

D:\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


0 32411678.0
1 31821254.0
2 36366470.0
3 39483470.0
4 35045570.0
5 23726128.0
6 12366000.0
7 5703575.0
8 2783157.0
9 1613963.5
10 1110176.9
11 852431.56
12 693252.25
13 579938.25
14 492466.06
15 422042.84
16 364013.9
17 315578.78
18 274822.3
19 240324.23
20 210928.53
21 185753.53
22 164100.47
23 145461.8
24 129312.5
25 115258.86
26 102981.18
27 92223.266
28 82776.18
29 74470.74
30 67121.17
31 60605.125
32 54817.645
33 49662.43
34 45060.53
35 40944.977
36 37255.773
37 33943.062
38 30963.555
39 28278.371
40 25854.418
41 23662.754
42 21678.625
43 19880.09
44 18249.746
45 16766.582
46 15417.279
47 14189.137
48 13068.486
49 12044.978
50 11108.893
51 10252.022
52 9467.367
53 8747.801
54 8087.2427
55 7480.502
56 6923.086
57 6410.367
58 5938.4224
59 5503.5776
60 5102.9863
61 4733.6255
62 4392.949
63 4078.127
64 3787.3838
65 3518.812
66 3270.28
67 3040.4014
68 2827.639
69 2630.7683
70 2448.345
71 2279.3318
72 2122.7354
73 1977.3933
74 1842.5115
75 1717.3005
76 1600.9949
77 1492.9688
78 1392.628

## nn 模块（neural network）

### PyTorch: nn

计算图和 autograd 是一个非常强大的用于定义复杂的运算符并自动微分的范例；然而，对于大型神经网络，原始 autograd 可能有点太低级。

在构建神经网络时，我们经常考虑将计算安排到层中，其中一些层具有可学习的参数，这些参数将在学习期间进行优化。

在 TensorFlow 中，像 Keras，TensorFlow-Slim 和 TFLearn 这样的软件包提供了对构建神经网络有用的原始计算图的更高级别的抽象。

在 PyTorch 中，nn 包服务于同样的目的。nn 包定义了一组模块，它们大致相当于神经网络层。模块接收输入 Tensor 并计算输出 Tensor，但也可以保持内部状态，例如包含可学习参数的 Tensor。nn 包还定义了一组在训练神经网络时常用的有用损失函数。

在这个例子中，我们使用nn包来实现我们的双层网络：

In [6]:
# -*- coding: utf-8 -*-
import torch

# N是数据样本数，D_in是输入层维度
# H是隐藏层维度，D_out是输出层维度
N, D_in, H, D_out = 64, 1000, 100, 10

# 创建随机输入和输出数据
x = torch.randn(N, D_in)
y = torch.randn(N, D_out)

# 使用 nn 将模型定义为神经网络层的序列
# nn.Sequential 是一个包含其他模块的模块，并按序应用模块以产生输出
# 每个线性模块使用线性函数由输入计算输出，并为其权重和偏置保留内部 Tensor

model = torch.nn.Sequential(
    torch.nn.Linear(D_in, H),
    torch.nn.ReLU(),
    torch.nn.Linear(H, D_out),
)

# nn 还包含了常用损失函数的定义
# 使用均方误差 MSE 作为我们的损失函数
loss_fn = torch.nn.MSELoss(reduction='sum')

learning_rate = 1e-4
iteration_count = 500  # 迭代次数
for t in range(iteration_count):
    # 前向传播：将 x 传递给 model 以计算预测值 y
    # 模块对象重载 __call__ 运算符，所以能直接像函数一样调用
    # 传递一个 Tensor 给模型，模型将输出一个含有数据的 Tensor
    y_pred = model(x)

    # 计算并打印损失
    # 将预测值与真实值传递给 loss_fn，返回损失
    loss = loss_fn(y_pred, y)
    print("iteration", t, ",", "loss:", loss.item())  # t是循环变量

    # 在计算反向传播前清零梯度
    model.zero_grad()

    # 反向传播：计算损失关于所有可学习参数的梯度
    # 在内部，每个模块的参数都存储在具有 requires_grad = True 的 Tensor 中
    # 因此该调用将计算模型中所有可学习参数的梯度
    loss.backward()

    # 使用梯度下降更新权重，每个参数是一个 Tensor
    with torch.no_grad():
        for param in model.parameters():
            param -= learning_rate * param.grad

iteration 0 , loss: 649.6320190429688
iteration 1 , loss: 602.88037109375
iteration 2 , loss: 562.5580444335938
iteration 3 , loss: 526.9771728515625
iteration 4 , loss: 495.0279541015625
iteration 5 , loss: 466.4129333496094
iteration 6 , loss: 440.5115661621094
iteration 7 , loss: 416.557861328125
iteration 8 , loss: 394.315673828125
iteration 9 , loss: 373.5847473144531
iteration 10 , loss: 354.14697265625
iteration 11 , loss: 335.86767578125
iteration 12 , loss: 318.6815185546875
iteration 13 , loss: 302.4425354003906
iteration 14 , loss: 286.96038818359375
iteration 15 , loss: 272.11285400390625
iteration 16 , loss: 257.91082763671875
iteration 17 , loss: 244.36141967773438
iteration 18 , loss: 231.4207763671875
iteration 19 , loss: 219.1044464111328
iteration 20 , loss: 207.30130004882812
iteration 21 , loss: 196.0321807861328
iteration 22 , loss: 185.2052459716797
iteration 23 , loss: 174.88754272460938
iteration 24 , loss: 165.01783752441406
iteration 25 , loss: 155.61906433105

iteration 211 , loss: 0.01221898477524519
iteration 212 , loss: 0.01173127070069313
iteration 213 , loss: 0.011263889260590076
iteration 214 , loss: 0.010815859772264957
iteration 215 , loss: 0.010386279784142971
iteration 216 , loss: 0.009974314831197262
iteration 217 , loss: 0.00957938376814127
iteration 218 , loss: 0.009200620464980602
iteration 219 , loss: 0.008837270550429821
iteration 220 , loss: 0.008489087224006653
iteration 221 , loss: 0.008155083283782005
iteration 222 , loss: 0.007834739051759243
iteration 223 , loss: 0.007527402136474848
iteration 224 , loss: 0.007232581730931997
iteration 225 , loss: 0.00694950670003891
iteration 226 , loss: 0.0066780513152480125
iteration 227 , loss: 0.006417642813175917
iteration 228 , loss: 0.006167740561068058
iteration 229 , loss: 0.005927667021751404
iteration 230 , loss: 0.005697357002645731
iteration 231 , loss: 0.005476396530866623
iteration 232 , loss: 0.0052645099349319935
iteration 233 , loss: 0.00506083806976676
iteration 234 

iteration 403 , loss: 1.1428570360294543e-05
iteration 404 , loss: 1.1057282790716272e-05
iteration 405 , loss: 1.0699145605030935e-05
iteration 406 , loss: 1.035140212479746e-05
iteration 407 , loss: 1.001528562483145e-05
iteration 408 , loss: 9.689575563243125e-06
iteration 409 , loss: 9.376405614602845e-06
iteration 410 , loss: 9.073151886695996e-06
iteration 411 , loss: 8.780773896432947e-06
iteration 412 , loss: 8.496646842104383e-06
iteration 413 , loss: 8.2215392467333e-06
iteration 414 , loss: 7.957321940921247e-06
iteration 415 , loss: 7.701451067987364e-06
iteration 416 , loss: 7.452852059941506e-06
iteration 417 , loss: 7.214126071630744e-06
iteration 418 , loss: 6.9803186306671705e-06
iteration 419 , loss: 6.757031314919004e-06
iteration 420 , loss: 6.5402305153838824e-06
iteration 421 , loss: 6.329547431960236e-06
iteration 422 , loss: 6.127862434368581e-06
iteration 423 , loss: 5.931303803663468e-06
iteration 424 , loss: 5.742279427067842e-06
iteration 425 , loss: 5.55845

### PyTorch: optim

到目前为止，我们通过手动改变持有可学习参数的 Tensor 来更新模型的权重（使用 torch.no_grad() 或 .data 以避免在 autograd 中跟踪历史记录）。 对于像随机梯度下降这样的简单优化算法来说，这不是一个巨大的负担，但在实践中，我们经常使用更复杂的优化器如 AdaGrad，RMSProp，Adam 等来训练神经网络。

PyTorch 中的 optim 包抽象出优化算法的思想，并提供常用优化算法的实现。

在这个例子中，我们将使用 nn 包像以前一样定义我们的模型，但我们将使用 optim 包提供的 Adam 算法优化模型：

In [7]:
# -*- coding: utf-8 -*-
import torch

# N是数据样本数，D_in是输入层维度
# H是隐藏层维度，D_out是输出层维度
N, D_in, H, D_out = 64, 1000, 100, 10

# 创建随机输入和输出数据
x = torch.randn(N, D_in)
y = torch.randn(N, D_out)

# 使用 nn 定义模型与损失函数
model = torch.nn.Sequential(
    torch.nn.Linear(D_in, H),
    torch.nn.ReLU(),
    torch.nn.Linear(H, D_out),
)
loss_fn = torch.nn.MSELoss(reduction='sum')

# 使用 optim 包定义一个优化器，它将为我们更新权重
# 此处使用 Adam
# optim 包包含许多其他优化算法
# Adam 构造函数的首个参数告诉优化器哪个 Tensor 应该被更新
learning_rate = 1e-4
iteration_count = 500  # 迭代次数
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

for t in range(iteration_count):
    # 前向传播：将 x 传递给 model 以计算预测值 y
    y_pred = model(x)

    # 计算并打印损失
    loss = loss_fn(y_pred, y)
    print("iteration", t, ",", "loss:", loss.item())  # t是循环变量

    # 在计算反向传播前清零梯度
    # 在反向传播之前，使用优化器对象将要更新的变量的所有梯度归零（这是模型的可学习权重）
    # 这是因为默认情况下，只要调用.backward()，梯度就会累积在缓冲区中（即不会被覆盖）
    # 有关更多详细信息，请查看 torch.autograd.backward 的文档
    model.zero_grad()

    # 反向传播：计算损失关于所有可学习参数的梯度
    loss.backward()

    # 调用优化器的 step 函数，更新参数
    optimizer.step()

iteration 0 , loss: 694.8057861328125
iteration 1 , loss: 677.6531372070312
iteration 2 , loss: 661.04931640625
iteration 3 , loss: 644.896484375
iteration 4 , loss: 629.1853637695312
iteration 5 , loss: 613.8985595703125
iteration 6 , loss: 599.0946044921875
iteration 7 , loss: 584.7273559570312
iteration 8 , loss: 570.8052368164062
iteration 9 , loss: 557.2130126953125
iteration 10 , loss: 543.9581298828125
iteration 11 , loss: 531.0118408203125
iteration 12 , loss: 518.3655395507812
iteration 13 , loss: 506.0689697265625
iteration 14 , loss: 494.1007995605469
iteration 15 , loss: 482.4564514160156
iteration 16 , loss: 471.1747741699219
iteration 17 , loss: 460.2269287109375
iteration 18 , loss: 449.57568359375
iteration 19 , loss: 439.17877197265625
iteration 20 , loss: 429.0790100097656
iteration 21 , loss: 419.29608154296875
iteration 22 , loss: 409.78472900390625
iteration 23 , loss: 400.469482421875
iteration 24 , loss: 391.3703918457031
iteration 25 , loss: 382.5168151855469
it

iteration 209 , loss: 0.697719156742096
iteration 210 , loss: 0.6672675609588623
iteration 211 , loss: 0.6381561160087585
iteration 212 , loss: 0.6102754473686218
iteration 213 , loss: 0.5836135745048523
iteration 214 , loss: 0.5581120848655701
iteration 215 , loss: 0.5337255597114563
iteration 216 , loss: 0.5104163289070129
iteration 217 , loss: 0.4880940318107605
iteration 218 , loss: 0.46675166487693787
iteration 219 , loss: 0.4463338255882263
iteration 220 , loss: 0.4268069267272949
iteration 221 , loss: 0.4081316888332367
iteration 222 , loss: 0.3902675211429596
iteration 223 , loss: 0.37318116426467896
iteration 224 , loss: 0.3568435609340668
iteration 225 , loss: 0.3412090539932251
iteration 226 , loss: 0.32625705003738403
iteration 227 , loss: 0.3119499087333679
iteration 228 , loss: 0.29826873540878296
iteration 229 , loss: 0.2851804494857788
iteration 230 , loss: 0.2727362811565399
iteration 231 , loss: 0.2608889937400818
iteration 232 , loss: 0.24956701695919037
iteration 23

iteration 400 , loss: 0.00011063663987442851
iteration 401 , loss: 0.00010564574040472507
iteration 402 , loss: 0.00010088491399073973
iteration 403 , loss: 9.634345769882202e-05
iteration 404 , loss: 9.201249486068264e-05
iteration 405 , loss: 8.787601836957037e-05
iteration 406 , loss: 8.393125608563423e-05
iteration 407 , loss: 8.016546780709177e-05
iteration 408 , loss: 7.65705481171608e-05
iteration 409 , loss: 7.313646347029135e-05
iteration 410 , loss: 6.986078369664028e-05
iteration 411 , loss: 6.673353345831856e-05
iteration 412 , loss: 6.374895019689575e-05
iteration 413 , loss: 6.089794260333292e-05
iteration 414 , loss: 5.81725180381909e-05
iteration 415 , loss: 5.55708866158966e-05
iteration 416 , loss: 5.3087038395460695e-05
iteration 417 , loss: 5.071537088952027e-05
iteration 418 , loss: 4.84485317429062e-05
iteration 419 , loss: 4.6283799747470766e-05
iteration 420 , loss: 4.421397534315474e-05
iteration 421 , loss: 4.223949144943617e-05
iteration 422 , loss: 4.0353148

### PyTorch: 自定义 nn 模块

有时，您需要指定**比现有模块序列更复杂**的模型; 对于这些情况，您可以通过子类化 nn.Module 定义自己的模块，并定义一个接收输入 Tensor 的 forward，并使用其他模块或 Tensor 上的其他 autograd 操作生成输出Tensor。

在这个例子中，我们将我们的双层网络实现为自定义 Module 子类：

In [8]:
# -*- coding: utf-8 -*-
import torch


class TwoLayerNet(torch.nn.Module):
    def __init__(self, D_in, H, D_out):
        """
        构造函数中，实例化两个 nn.Linear 模块并将其指定为成员变量
        """
        super(TwoLayerNet, self).__init__()
        self.linear1 = torch.nn.Linear(D_in, H)
        self.linear2 = torch.nn.Linear(H, D_out)

    def forward(self, x):
        """
        前向函数中，接受一个输入数据 Tensor 且必须返回一个输出数据 Tensor
        可以使用构造函数中定义的模块或者 Tensor 的任意运算。
        """
        h_relu = self.linear1(x).clamp(min=0)
        y_pred = self.linear2(h_relu)
        return y_pred


# N是数据样本数，D_in是输入层维度
# H是隐藏层维度，D_out是输出层维度
N, D_in, H, D_out = 64, 1000, 100, 10

# 创建随机输入和输出数据
x = torch.randn(N, D_in)
y = torch.randn(N, D_out)

# 构造模型
model = TwoLayerNet(D_in, H, D_out)

# 构造损失函数和一个优化器
# SGD 构造器中的 model.parameters() 包含两层 nn.linear 模块中的可学习参数
# 它们是模型的成员

criterion = torch.nn.MSELoss(reduction='sum')
optimizer = torch.optim.SGD(model.parameters(), lr=1e-4)

for t in range(500):
    # 前向传播
    y_pred=model(x)
    
    # 计算并打印损失
    loss = criterion(y_pred,y)
    print(t,loss.item())
    
    # 清零梯度，执行反向传播，更新权重
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

0 650.4168090820312
1 604.8271484375
2 565.3889770507812
3 530.7918701171875
4 499.9839782714844
5 472.0627136230469
6 446.8885192871094
7 423.9190673828125
8 402.59405517578125
9 382.6856994628906
10 363.968017578125
11 346.40966796875
12 329.8550720214844
13 314.1585998535156
14 299.1746520996094
15 284.8345642089844
16 271.18603515625
17 258.0657043457031
18 245.41969299316406
19 233.26780700683594
20 221.6000213623047
21 210.37281799316406
22 199.6201934814453
23 189.3350830078125
24 179.4701385498047
25 170.0068359375
26 160.9403839111328
27 152.27197265625
28 143.979736328125
29 136.0373992919922
30 128.4136505126953
31 121.13982391357422
32 114.2225341796875
33 107.66429901123047
34 101.43965911865234
35 95.55449676513672
36 89.99089813232422
37 84.69075775146484
38 79.68389129638672
39 74.96304321289062
40 70.51262664794922
41 66.32493591308594
42 62.37954330444336
43 58.6663703918457
44 55.167930603027344
45 51.85204315185547
46 48.73835372924805
47 45.81789779663086
48 43.076

375 0.000146143909660168
376 0.00014181379810906947
377 0.00013760817819274962
378 0.0001335300476057455
379 0.00012957108265254647
380 0.00012573768617585301
381 0.00012201550271129236
382 0.00011841287050629035
383 0.00011491759505588561
384 0.00011152285878779367
385 0.00010822912736330181
386 0.00010503189696464688
387 0.00010193906200584024
388 9.892875823425129e-05
389 9.601411147741601e-05
390 9.318890079157427e-05
391 9.044264152180403e-05
392 8.778471237746999e-05
393 8.51990407682024e-05
394 8.269827958429232e-05
395 8.02657232270576e-05
396 7.791052485117689e-05
397 7.562558312201872e-05
398 7.340908632613719e-05
399 7.125573029043153e-05
400 6.916530401213095e-05
401 6.713885522913188e-05
402 6.517479778267443e-05
403 6.326658331090584e-05
404 6.141555786598474e-05
405 5.961869828752242e-05
406 5.78762155782897e-05
407 5.618750583380461e-05
408 5.4547541367355734e-05
409 5.295498704072088e-05
410 5.1408293074928224e-05
411 4.991134483134374e-05
412 4.845573494094424e-05
413

### PyTorch: 控制流与权重共享

作为动态图和权重共享的一个例子，我们现在实现一个非常奇怪的模型：一个完全连接的 ReLU 网络，在每个前向传播中选择 1 到 4 之间的随机数并使用那么多隐藏层，重复使用相同的权重多次以计算最里面的隐藏层。

对于这个模型，我们可以使用普通的 Python 流控制来实现循环，并且我们可以通过在定义前向传播时多次重复使用相同的模块来实现最内层之间的权重共享。

我们可以轻松地将此模型实现为 Module 子类：

In [9]:
# -*- coding: utf-8 -*-
import random
import torch


class DynamicNet(torch.nn.Module):
    def __init__(self, D_in, H, D_out):
        """
        构造 nn.Linear 的三个实例
        """
        super(DynamicNet, self).__init__()
        self.input_linear = torch.nn.Linear(D_in, H)
        self.middle_linear = torch.nn.Linear(H, H)
        self.output_linear = torch.nn.Linear(H, D_out)

    def forward(self, x):
        """   
        对于模型的前向传播，我们随机选择 0,1,2或3，并多次重用 middle_linear 模块来计算隐藏层表示
        由于每个前向传播都构建了一个动态计算图，因此在定义模型的前向传播时，我们可以使用普通的 Python 控制流操作符，如循环或条件语句
        在这里，我们还看到在定义计算图时多次重复使用相同的模块是完全安全的
        这是对 Lua Torch 的一项重大改进，它的每个模块只能使用一次
        """
        h_relu = self.input_linear(x).clamp(min=0)
        for _ in range(random.randint(0, 4)):  # 0 ~ 3
            h_relu = self.middle_linear(h_relu).clamp(min=0)
        y_pred = self.output_linear(h_relu)
        return y_pred


# N是数据样本数，D_in是输入层维度
# H是隐藏层维度，D_out是输出层维度
N, D_in, H, D_out = 64, 1000, 100, 10

# 创建随机输入和输出数据
x = torch.randn(N, D_in)
y = torch.randn(N, D_out)

# 创建随机输入和输出数据
x = torch.randn(N, D_in)
y = torch.randn(N, D_out)

# 构造模型
model = DynamicNet(D_in, H, D_out)

# 构造损失函数和一个优化器
# 使用 SGD 直接训练这个奇怪的模型有些困难，所以我们加上动量
criterion = torch.nn.MSELoss(reduction='sum')
optimizer = torch.optim.SGD(model.parameters(), lr=1e-4, momentum=0.9)

for t in range(500):
    # 前向传播
    y_pred = model(x)

    # 计算并打印损失
    loss = criterion(y_pred, y)
    print(t, loss.item())

    # 清零梯度，执行反向传播，更新权重
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

0 684.6495971679688
1 646.7960205078125
2 646.8902587890625
3 641.616455078125
4 642.1076049804688
5 640.3995361328125
6 639.8314208984375
7 629.090576171875
8 636.523193359375
9 439.0577697753906
10 601.0468139648438
11 618.2501831054688
12 584.3655395507812
13 610.7720336914062
14 630.1234741210938
15 635.0167236328125
16 532.8248901367188
17 300.41424560546875
18 497.7484130859375
19 573.45263671875
20 626.7001342773438
21 549.343017578125
22 616.338134765625
23 515.1063232421875
24 565.800048828125
25 471.1210021972656
26 339.4145202636719
27 319.05706787109375
28 400.5921630859375
29 502.4747619628906
30 259.2665100097656
31 461.2328186035156
32 203.297119140625
33 422.7790832519531
34 369.6379699707031
35 341.1153869628906
36 149.56039428710938
37 257.345458984375
38 164.8937225341797
39 276.5129699707031
40 147.45355224609375
41 203.12112426757812
42 251.89442443847656
43 264.73565673828125
44 108.53075408935547
45 99.23666381835938
46 117.24641418457031
47 898.188232421875
48 4

375 25.092817306518555
376 20.329994201660156
377 13.308563232421875
378 12.667623519897461
379 5.371011257171631
380 6.465823650360107
381 10.244417190551758
382 15.709258079528809
383 20.36738395690918
384 7.933638572692871
385 3.0845744609832764
386 19.895034790039062
387 6.562630653381348
388 5.080763816833496
389 3.2361795902252197
390 14.162188529968262
391 4.230726718902588
392 4.042773723602295
393 4.330113410949707
394 3.5533559322357178
395 7.274179935455322
396 3.703739643096924
397 3.9915847778320312
398 3.303746461868286
399 3.1808230876922607
400 7.0486273765563965
401 11.859415054321289
402 4.366374492645264
403 5.8661603927612305
404 9.36649227142334
405 13.153993606567383
406 2.571829080581665
407 1.5467166900634766
408 3.5508220195770264
409 17.692949295043945
410 9.411348342895508
411 1.8874576091766357
412 4.455809593200684
413 7.347081184387207
414 2.688293933868408
415 2.2896595001220703
416 21.945087432861328
417 4.9614105224609375
418 5.222107887268066
419 9.541